In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1616024691518_0002,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@451376b1


In [2]:
import org.apache.spark.sql.{DataFrame,Column}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.{DataFrame, Column}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._


In [42]:
def saveCSV(df: DataFrame, path: String): Unit =
(
    df
    .repartition(1)
    .write
    .option("header", "true")
    .csv(path)
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

saveCSV: (df: org.apache.spark.sql.DataFrame, path: String)Unit


In [30]:
val s3_path = "s3://trab4-bucket/datasets/torre-eifel.parquet"

val df = spark.read.parquet(s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3_path: String = s3://trab4-bucket/datasets/torre-eifel.parquet
df: org.apache.spark.sql.DataFrame = [text: string, title: string ... 2 more fields]


In [32]:
df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+----------+
|                text|               title|                  id|      Date|
+--------------------+--------------------+--------------------+----------+
|This is the most ...|Must do even it w...|5921cdae4b679c464...|2017-05-20|
|My significant ot...|           A Classic|5921cdae4b679c464...|2017-05-20|
|We had a tour to ...|         Wet weather|5921cdae4b679c464...|2017-05-20|
|Visited with my w...|    Not to be missed|5921cdae4b679c464...|2017-05-20|
|We went in the ni...|          Awesome!!!|5921cdae4b679c464...|2017-05-20|
|Dont hesitate and...|Iconic place for ...|5921cdbf4b679c464...|2017-05-20|
|I enjoyed the tow...|          Delightful|5921cdbf4b679c464...|2017-05-20|
|Read through the ...|Wonderful experie...|5921cdbf4b679c464...|2017-05-20|
|This by far was o...|                 Wow|5921cdbf4b679c464...|2017-05-20|
|Something you hav...|              Great |5921cdcf4b679c464...|2017-05-19|
+-----------

# 2 a) Encontre as palavras mais utilizadas nas avaliações.

In [49]:
val dfWords = (
    df.withColumn("word", explode(split(col("text")," ")))
    .withColumn("word",lower(col("word")))
    .filter(length(col("word")) > 4)
    .groupBy("word") 
    .count() 
    .orderBy(desc("count"))
    .limit(15)
)

dfWords.show()

saveCSV(dfWords,"s3://trab4-bucket/datasets/questoes/2/a.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfWords: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [word: string, count: bigint]
+-------+-----+
|   word|count|
+-------+-----+
|  tower| 4184|
| eiffel| 3246|
|  there| 2049|
|  paris| 1997|
|  visit| 1322|
|  views| 1239|
|tickets| 1222|
|  worth| 1164|
|  night| 1107|
| tower.| 1039|
|  great| 1038|
|  first|  945|
| around|  840|
| second|  835|
| paris.|  831|
+-------+-----+



# 2 b) Encontre as expressões mais usadas.

In [34]:
def splitOnLength(len: Int) = udf(
    (str: String) => {
  str.toLowerCase.grouped(len).toSeq
})


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitOnLength: (len: Int)org.apache.spark.sql.expressions.UserDefinedFunction


In [50]:
val threshold = 45
val dfExpr = (
    df
    .na.fill(Map("text" -> " "))
    .withColumn("Expr",splitOnLength(threshold)(col("text")))
    .withColumn("Expr", explode(col("Expr")))
    .filter(
        length(col("Expr")) === threshold
    )
    .groupBy("Expr")
    .count()
    .orderBy(desc("count"))
    .limit(15)
)

dfExpr.show(false)

saveCSV(dfExpr,"s3://trab4-bucket/datasets/questoes/2/b.csv")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

threshold: Int = 45
dfExpr: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Expr: string, count: bigint]
+---------------------------------------------+-----+
|Expr                                         |count|
+---------------------------------------------+-----+
|no trip to paris would be complete without a |3    |
|you can't go to paris and not visit the eiffe|3    |
|you can't go to paris without visiting the ei|3    |
|no visit to paris would be complete without a|2    |
|we got there first thing in the morning which|2    |
|a trip to paris is not complete without a vis|2    |
|you cannot go to paris without visiting the e|2    |
|the eiffel tower is to paris what the statue |2    |
|of liberty is to new york and what big ben is|2    |
|well what can i say that hasn't already been |2    |
|this was my second visit to the eiffel tower.|2    |
|the eiffel tower can be seen from all over pa|2    |
|so worth climbing up rather than taking the e|2    |
|we loved our stay in

# 2 c) Encontre os principais tópicos relacionados às revisões.

In [51]:
val threshold_max = 16
val threshold_min = 3
val dfTopics = (
    df.withColumn("word", explode(split(col("title")," ")))
    .withColumn("word",lower(col("word")))
    .filter(length(col("word")) < threshold_max && length(col("word")) > threshold_min)
    .groupBy("word") 
    .count() 
    .orderBy(desc("count"))
    .limit(15)
)

dfTopics.show(false)

saveCSV(dfTopics,"s3://trab4-bucket/datasets/questoes/2/c.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

threshold_max: Int = 16
threshold_min: Int = 3
dfTopics: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [word: string, count: bigint]
+----------+-----+
|word      |count|
+----------+-----+
|must      |687  |
|paris     |634  |
|eiffel    |579  |
|tower     |556  |
|amazing   |426  |
|view      |340  |
|beautiful |322  |
|great     |317  |
|night     |288  |
|visit     |287  |
|views     |239  |
|worth     |219  |
|iconic    |181  |
|from      |148  |
|experience|136  |
+----------+-----+



# 2 d) Mapeie a distribuição temporal das revisões.

In [53]:
val dfTimeSeries = (df
                    .groupBy("Date")
                    .count()
                    .orderBy(asc("Date"))
                   )

dfTimeSeries.show(false)

saveCSV(dfTimeSeries,"s3://trab4-bucket/datasets/questoes/2/d.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfTimeSeries: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Date: date, count: bigint]
+----------+-----+
|Date      |count|
+----------+-----+
|2015-07-27|16   |
|2015-07-28|16   |
|2015-07-29|5    |
|2015-07-30|3    |
|2015-07-31|11   |
|2015-08-01|5    |
|2015-08-02|9    |
|2015-08-03|14   |
|2015-08-04|3    |
|2015-08-05|6    |
|2015-08-06|16   |
|2015-08-07|10   |
|2015-08-08|16   |
|2015-08-09|15   |
|2015-08-10|25   |
|2015-08-11|25   |
|2015-08-12|15   |
|2015-08-13|37   |
|2015-08-14|34   |
|2015-08-15|18   |
+----------+-----+
only showing top 20 rows

